# IoTアプリケーションの開発効率の向上を目的とした開発者によるコードの変更を動的に適用する方式の提案と実装

このノートブックは、情報処理学会の[第207回SE研究発表会](http://www.ipsj.or.jp/kenkyukai/event/se207.html)において口頭発表予定の研究報告論文「IoTアプリケーションの開発効率の向上を目的とした開発者によるコードの変更を動的に適用する方式の提案と実装」のための実験について詳細を記録・公開するためのリポジトリである。

## 更新対象とするIoTアプリケーション

各方式の比較検討に用いるアプリケーションは、本リポジトリの[target_app](./target_app)ディレクトリに収められている、IoTデバイス上で動作するEchoサーバが更新対象のアプリケーションである。

## 比較検討する方式

上記のIoTアプリケーションコードの変更をIoTデバイスに適用する方式として、以下の3つについて比較検討する。

1. ファームウェアイメージの全体を適用する方式
2. ファームウェアイメージの差分を適用する方式
3. アプリケーションコードを動的に適用する方式（提案方式）

それぞれの詳細については、研究報告論文を参照されたい。

## 比較検討の方法

各方式を比較するための指標として、開発者によるコードの変更をデバイスに適用し、更新対象のIoTアプリケーション（Echoサーバ）がTCP通信可能となるまでの時間を用いる。

方式を実装するためのIoTデバイス開発プラットフォームとして、[Nerves](https://www.nerves-project.org/)を用いる。筆者らによる提案方式3.の実装を加えると、上記3つの方式をすべて実装しているのがNervesだからである。ハードウェアとしては、Raspberry Piを用いる。

方式1.および2.については、スクリプト[measure.sh](./measure.sh)を用いて、以下の時間をそれぞれ計測する。

1. ファームウェアイメージの生成（`time(1)`で計測）
2. ファームウェアイメージのデバイスへの送信および適用（`time(1)`で計測）
3. デバイスの再起動（`ping(1)`で通信が疎通するまでの時間を計測。タイムアウトを100msに設定してループ）
4. IoTアプリケーションの起動（`necat(1)`でTCP通信が疎通するまでの時間を計測。タイムアウトを1msに設定してループ(※)）

※ ただし、実際には1msの粒度ではループしない。下記の実験環境においては、短くても10ms程度にとどまるようであった。

方式3.ではIoTデバイスの再起動は必要ないため、以下に要する時間を計測する。

1. コードの変更のデバイスへの適用（`time(1)`で計測）
2. IoTアプリケーションの再起動（`necat(1)`でTCP通信が疎通するまでの時間を計測）

3回計測を行い、それらの平均を用いる。

また、それぞれの方式について、アップデート対象となるファームウェアあるいはコードの容量についても付記する。

## 実験環境

### 開発環境

* Model: MacBook Pro 13-inch, 2018
* CPU: 2.7 GHz Quad-Core Intel Core i7
* Memory: 16 GB 2133 MHz LPDDR3

### IoTデバイス

* Raspberry Pi 3 Model B
* 開発環境との通信: 有線LAN
* プラットフォーム: Nerves 1.7.2

このNotebookでは、Nervesの開発環境の整備については解説しない。「[ElixirでIoT#4.1：Nerves開発環境の準備（2020年11月版）](https://qiita.com/takasehideki/items/88dda57758051d45fcf9)」等の記事を参照されたい。

## 実験

実験では、筆者らが用意したスクリプト[measure.sh](./measure.sh)を用いる。

### 1. ファームウェアイメージ全体の適用

以下に要する時間を計測する。

1. ファームウェアイメージ全体の生成（`time(1)`で計測）
2. ファームウェアイメージのデバイスへの送信と適用（`time(1)`で計測）
3. デバイスの再起動（`ping(1)`で通信が疎通するまでの時間を計測）
4. IoTアプリケーションの起動（`necat(1)`でTCP通信が疎通するまでの時間を計測）

3回計測を行い、それらの平均を用いる。

#### 1回目

In [23]:
!cd target_app && ../measure.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m28.831s
user	0m6.262s
sys	0m2.410s

Time to upload firmware

real	0m14.545s
user	0m5.175s
sys	0m1.254s

Time to reboot

time: 22.380 sec.

Time to accept

time: .547 sec. (timeout: 14)



ビルドされたファームウェアのサイズ:

In [24]:
!ls -lh target_app/_build/rpi3_dev/nerves/images/target_app.fw

-rw-r--r--  1 antipop  2033490572    42M  1 21 14:37 target_app/_build/rpi3_dev/nerves/images/target_app.fw


#### 2回目

In [27]:
!cd target_app && ../measure.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m31.347s
user	0m6.423s
sys	0m2.363s

Time to upload firmware

real	0m14.893s
user	0m4.900s
sys	0m1.313s

Time to reboot

time: 22.492 sec.

Time to accept

time: .676 sec. (timeout: 21)



ビルドされたファームウェアのサイズ:

In [28]:
!ls -lh target_app/_build/rpi3_dev/nerves/images/target_app.fw

-rw-r--r--  1 antipop  2033490572    42M  1 21 14:42 target_app/_build/rpi3_dev/nerves/images/target_app.fw


#### 3回目

In [31]:
!cd target_app && ../measure.sh firmware

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware)

real	0m28.187s
user	0m6.467s
sys	0m2.357s

Time to upload firmware

real	0m14.045s
user	0m5.217s
sys	0m1.264s

Time to reboot

time: 22.666 sec.

Time to accept

time: .681 sec. (timeout: 21)



ビルドされたファームウェアのサイズ:

In [32]:
!ls -lh target_app/_build/rpi3_dev/nerves/images/target_app.fw

-rw-r--r--  1 antipop  2033490572    42M  1 21 14:49 target_app/_build/rpi3_dev/nerves/images/target_app.fw


----

#### 方式（1）のまとめ

In [35]:
print("（1） {:.2f}秒".format((28.831+31.347+28.187)/3))
print("（2） {:.2f}秒".format((14.545+14.893+14.045)/3))
print("（3） {:.2f}秒".format((22.380+22.492+22.666)/3))
print("（4） {:.2f}秒".format((0.547+0.676+0.681)/3))

（1） 29.45秒
（2） 14.49秒
（3） 22.51秒
（4） 0.63秒


また、ビルドされたファームウェアのサイズは`42MB`であった。

----

### 2. ファームウェアイメージ差分の適用

以下に要する時間を計測する。

1. ファームウェアイメージ差分の生成（`time(1)`で計測）
2. ファームウェアイメージ差分のデバイスへの送信と適用（`time(1)`で計測）
3. デバイスの再起動（`ping(1)`で通信が疎通するまでの時間を計測）
4. IoTアプリケーションの起動（`necat(1)`でTCP通信が疎通するまでの時間を計測）

3回計測を行い、それらの平均を用いる。

#### 1回目

In [25]:
!cd target_app && ../measure.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m28.464s
user	0m6.116s
sys	0m2.383s

Time to upload firmware

real	0m16.972s
user	0m3.800s
sys	0m1.012s

Time to reboot

time: 22.462 sec.

Time to accept

time: .496 sec. (timeout: 15)



ビルドされたファームウェア差分のサイズ:

In [26]:
!ls -lh target_app/_build/rpi3_dev/nerves/images/patch.fw

-rw-r--r--  1 antipop  2033490572   6.0M  1 21 14:40 target_app/_build/rpi3_dev/nerves/images/patch.fw


#### 2回目

In [29]:
!cd target_app && ../measure.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m32.797s
user	0m6.526s
sys	0m2.361s

Time to upload firmware

real	0m17.886s
user	0m4.030s
sys	0m1.170s

Time to reboot

time: 22.465 sec.

Time to accept

time: .507 sec. (timeout: 15)



ビルドされたファームウェア差分のサイズ:

In [30]:
!ls -lh target_app/_build/rpi3_dev/nerves/images/patch.fw

-rw-r--r--  1 antipop  2033490572   6.0M  1 21 14:48 target_app/_build/rpi3_dev/nerves/images/patch.fw


#### 3回目

In [33]:
!cd target_app && ../measure.sh firmware.patch

nerves.local is at 172.31.68.221

Time to build firmware (mix firmware.patch)

real	0m29.017s
user	0m6.282s
sys	0m2.338s

Time to upload firmware

real	0m17.513s
user	0m3.914s
sys	0m1.132s

Time to reboot

time: 22.693 sec.

Time to accept

time: .620 sec. (timeout: 19)



ビルドされたファームウェア差分のサイズ:

In [34]:
!ls -lh target_app/_build/rpi3_dev/nerves/images/patch.fw

-rw-r--r--  1 antipop  2033490572   6.0M  1 21 14:51 target_app/_build/rpi3_dev/nerves/images/patch.fw


----

#### 方式（2）のまとめ

In [36]:
print("（1） {:.2f}秒".format((28.464+32.797+29.017)/3))
print("（2） {:.2f}秒".format((16.972+17.886+17.513)/3))
print("（3） {:.2f}秒".format((22.462+22.465+22.693)/3))
print("（4） {:.2f}秒".format((0.496+0.507+0.620)/3))

（1） 30.09秒
（2） 17.46秒
（3） 22.54秒
（4） 0.54秒


また、ビルドされたファームウェア差分のサイズは`6MB`であった。

----

### 3. アプリケーションコードの部分適用

この方式ではIoTデバイスの再起動は必要ないため、以下に要する時間を計測する。

1. コードの変更のデバイスへの適用（`time(1)`で計測する）
4. IoTアプリケーションと`nc(1)`でTCP通信が疎通すること

3回計測を行い、それらの平均を用いる。

#### 1回目

In [38]:
!cd target_app && ../measure.sh upload.hotswap

nerves.local is at 172.31.68.221

Time to hotswap (mix upload.hotswap)

real	0m3.279s
user	0m4.450s
sys	0m1.286s

Time to accept

time: .042 sec. (timeout: 0)



#### 2回め

In [39]:
!cd target_app && ../measure.sh upload.hotswap

nerves.local is at 172.31.68.221

Time to hotswap (mix upload.hotswap)

real	0m3.342s
user	0m4.769s
sys	0m1.376s

Time to accept

time: .038 sec. (timeout: 0)



#### 3回目

In [40]:
!cd target_app && ../measure.sh upload.hotswap

nerves.local is at 172.31.68.221

Time to hotswap (mix upload.hotswap)

real	0m3.293s
user	0m4.630s
sys	0m1.332s

Time to accept

time: .037 sec. (timeout: 0)



#### 方式（3）のまとめ

In [42]:
print("（1） {:.2f}秒".format((3.279+3.342+3.293)/3))
print("（2） {:.2f}秒".format((0.042+0.038+0.037)/3))

（1） 3.30秒
（2） 0.04秒


上記の方式2については、IoTアプリケーションの起動プロセスを検知することができなかったという結論となる。

> timeout: 0

と表示されている通り、通信断が発生していることを検知できていないからである。平均0.04秒かかっているのは、計測コマンド（`ncat(1)`）の実行とTCP通信に関するオーバヘッドが計測されている。

### 各方式の計測結果のまとめ

各方式について、上記から以下の通り結果をまとめた。

|                     | Firmware Build | Firmware Deployment | Code Deployment | Device Reboot | Application Reboot | Total | Rate  |
|---------------------|---------------:|--------------------:|----------------:|--------------:|-------------------:|------:|------:|
| Method (1)          |          29.45 |               14.49 |               - |         22.51 |               0.63 | 66.88 |     1 |
| Method (2)          |          30.09 |               17.46 |               - |         22.54 |               0.54 | 70.63 |  1.05 |
| Proposed Method (3) |              - |                   - |            3.30 |             - |                  N/A |  3.30 |  0.05 |

----

## 方式（3）についての追加実験

上記の方式（3）は，IoTデバイスの再起動をともなわない方式である．一方で，Erlang VM上でのコードの更新は行われるため，以下の2つの面で通信の遮断が発生するかどうかを確認する．

1. 適用前後のデバイスとの通信を`ping(1)`で検証する
2. 適用前後のIoTアプリケーションとの通信を`nc(1)`で検証する

結論としては、1.については遮断は発生しない。2.については発生する。

以下に詳細を述べる。

### 1. 適用前後のデバイスとの通信を`ping(1)`で検証する

上記の適用を行う前後で、通信の遮断が発生するかどうかを、`ping(1)`コマンドにより検証する。

（1）ターミナルを2つ開いておき、一方では以下のコマンドを用いて、`nerves.local`に対して0.1秒間隔で`ping(1)`コマンドを実行する。

```
ping -i 0.1 nerves.local | while read pi; do echo "$(date '+[%Y/%m/%d %H:%M:%S]') $pi"; done
```

（2）他方では、上述の`mix upload.hotswap`コマンドを`date(1)`コマンドを前後に挟むことで時間を表示して実行する。

#### （1）の結果

（1）の結果は以下の通りである。

```
$ ping -i 0.1 nerves.local | while read pi; do echo "$(date '+[%Y/%m/%d %H:%M:%S]') $pi"; done
[2021/01/15 16:54:24] PING nerves.local (172.31.68.221): 56 data bytes
[2021/01/15 16:54:24] 64 bytes from 172.31.68.221: icmp_seq=0 ttl=64 time=0.726 ms
[2021/01/15 16:54:24] 64 bytes from 172.31.68.221: icmp_seq=1 ttl=64 time=0.651 ms
[2021/01/15 16:54:24] 64 bytes from 172.31.68.221: icmp_seq=2 ttl=64 time=0.833 ms
[2021/01/15 16:54:24] 64 bytes from 172.31.68.221: icmp_seq=3 ttl=64 time=0.920 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=4 ttl=64 time=0.977 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=5 ttl=64 time=0.668 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=6 ttl=64 time=0.840 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=7 ttl=64 time=0.832 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=8 ttl=64 time=0.783 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=9 ttl=64 time=0.774 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=10 ttl=64 time=0.858 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=11 ttl=64 time=0.821 ms
[2021/01/15 16:54:25] 64 bytes from 172.31.68.221: icmp_seq=12 ttl=64 time=0.865 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=13 ttl=64 time=0.731 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=14 ttl=64 time=0.770 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=15 ttl=64 time=0.624 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=16 ttl=64 time=0.719 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=17 ttl=64 time=0.810 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=18 ttl=64 time=0.685 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=19 ttl=64 time=0.781 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=20 ttl=64 time=0.742 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=21 ttl=64 time=0.816 ms
[2021/01/15 16:54:26] 64 bytes from 172.31.68.221: icmp_seq=22 ttl=64 time=0.583 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=23 ttl=64 time=0.547 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=24 ttl=64 time=0.618 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=25 ttl=64 time=0.592 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=26 ttl=64 time=0.556 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=27 ttl=64 time=0.525 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=28 ttl=64 time=0.498 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=29 ttl=64 time=0.506 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=30 ttl=64 time=0.504 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=31 ttl=64 time=0.514 ms
[2021/01/15 16:54:27] 64 bytes from 172.31.68.221: icmp_seq=32 ttl=64 time=0.558 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=33 ttl=64 time=0.556 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=34 ttl=64 time=0.525 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=35 ttl=64 time=0.559 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=36 ttl=64 time=0.547 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=37 ttl=64 time=0.517 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=38 ttl=64 time=0.518 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=39 ttl=64 time=0.534 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=40 ttl=64 time=0.617 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=41 ttl=64 time=0.530 ms
[2021/01/15 16:54:28] 64 bytes from 172.31.68.221: icmp_seq=42 ttl=64 time=0.581 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=43 ttl=64 time=0.633 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=44 ttl=64 time=0.534 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=45 ttl=64 time=0.533 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=46 ttl=64 time=0.548 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=47 ttl=64 time=0.520 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=48 ttl=64 time=0.561 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=49 ttl=64 time=0.490 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=50 ttl=64 time=0.524 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=51 ttl=64 time=0.501 ms
[2021/01/15 16:54:29] 64 bytes from 172.31.68.221: icmp_seq=52 ttl=64 time=0.639 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=53 ttl=64 time=0.735 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=54 ttl=64 time=0.771 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=55 ttl=64 time=0.815 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=56 ttl=64 time=0.841 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=57 ttl=64 time=0.777 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=58 ttl=64 time=0.701 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=59 ttl=64 time=0.688 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=60 ttl=64 time=0.596 ms
[2021/01/15 16:54:30] 64 bytes from 172.31.68.221: icmp_seq=61 ttl=64 time=0.755 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=62 ttl=64 time=0.586 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=63 ttl=64 time=0.543 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=64 ttl=64 time=0.552 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=65 ttl=64 time=0.760 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=66 ttl=64 time=0.844 ms
[2021/01/15 16:54:31] 64 bytes from 172.31.68.221: icmp_seq=67 ttl=64 time=0.870 ms
^C
```

#### （2）の結果

（2）の結果は以下の通りである。

```
$ date '+[%Y/%m/%d %H:%M:%S]'; mix upload.hotswap; date '+[%Y/%m/%d %H:%M:%S]'
[2021/01/15 16:54:26]
==> nerves
make[1]: Nothing to be done for `all'.
==> target_app

Nerves environment
  MIX_TARGET:   rpi3
  MIX_ENV:      dev

Compiling 2 files (.ex)
Generated target_app app
Successfully connected to hot_upload_test@nerves.local
Successfully deployed Elixir.TargetApp to hot_upload_test@nerves.local
Successfully deployed Elixir.TargetApp.Application to hot_upload_test@nerves.local
[2021/01/15 16:54:29]
```

よって、（1）の`ping(1)`の実行中に、（2）のコマンド実行による通信の切断は見られなかった。

### 2. 適用前後のIoTアプリケーションとの通信を`nc(1)`で検証する

上記の適用を行う前後で、IoTアプリケーションとの通信の遮断が発生するかどうかを、`nc(1)`コマンドにより検証する。

（1）ターミナルを2つ開いておき、一方では`nc nerves.local 9849`としてIoTアプリケーションと接続しておく。
（2）他方では、上述の`mix upload.hotswap`コマンドを実行する。

（1）を実行し、以下の通り入力を待ち受ける（コネクションを張ったままの）状態にしておく。

```
nc nerves.local 9849
test1
> test1
test2
> test2
```

その状態で（2）を実行すると、接続が切断される。これは、古いコードを参照しているプロセスをErlang VMが終了させるためである（参照: [Erlang -- Compilation and Code Loading](https://erlang.org/doc/reference_manual/code_loading.html#code-replacement)）。

したがって、適用前後では、IoTアプリケーションとの通信は切断される。